In [290]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable


# Imports

In [291]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from catboost.utils import eval_metric
from catboost import CatBoostRegressor, Pool

# Funcao de erro
from sklearn.metrics import mean_tweedie_deviance


import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from pathlib import Path

# Modelos para classificar se eh zero ou nao
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Modelos para predizer o valor de risco
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import statsmodels.api as sm

In [292]:
from ydata_profiling import ProfileReport

# Load data VSCODE

In [ ]:
# Importa o arquivo CSV 'train.csv' e carrega os dados em um DataFrame chamado df.
df = pd.read_csv('train.csv')

# Exibe as primeiras linhas do DataFrame para uma visualização inicial dos dados.
df

,CHAVE,TIPO_PROD,COBERTURA,COD_MODELO,ANO_MODELO,COD_TARIF,REGIAO,TIPO_FRANQ,VAL_FRANQ,IS_CASCO,CLAS_BONUS,SEXO,DATA_NASC,TEMPO_HAB,UTILIZACAO,CEP_PER,EXPOSICAO,OCORRENCIAS,INDENIZACAO
0,2870169,1,1,001228-9,2007,10,9,2,3692.0,20121.0,9,M,19600126,0,2,12216000,0.295890,0,0.0
1,479743,1,1,014094-5,2018,10,11,2,2681.0,70675.0,9,F,19850328,0,2,1231000,0.345205,0,0.0
2,2201308,1,1,004347-8,2010,10,11,2,3030.0,23208.0,0,M,19390511,0,2,9720000,0.309589,0,0.0
3,3132957,1,1,005190-0,2008,10,2,1,1546.0,19373.0,9,M,19730714,0,2,95330000,0.534247,0,0.0
4,3474671,2,1,015095-9,2017,10,21,1,2549.0,56783.0,7,M,19971002,0,1,46430000,0.572603,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3033103,3971983,2,1,999999-9,2019,23,2,1,3899.0,142301.0,0,F,19791231,0,0,96180000,0.400000,0,0.0
3033104,2901054,2,1,001461-3,2019,10,20,1,1718.0,33937.0,7,F,19800529,0,1,29018000,0.128767,1,7991.0
3033105,2820666,1,1,022112-0,2013,17,11,1,5921.0,124514.0,9,F,19550202,0,2,4116000,0.172603,0,0.0
3033106,4297186,1,1,025054-6,2014,10,2,1,1890.0,20742.0,5,F,19820218,0,2,97050000,0.553425,0,0.0


Para esse trabalho utilizamos os dados externos dos preços médios referentes aos códigos da tabela fipe. Segue o link do repositório com os dados: [tabela-fipe](https://www.kaggle.com/datasets/franckepeixoto/tabela-fipe)

O código carrega dados de dois arquivos CSV e realiza operações de visualização nas amostras desses dados.

1. **Leitura do Arquivo 'train.csv'**:
   - O arquivo 'train.csv' é lido usando o `pd.read_csv()` e armazenado em um DataFrame chamado `df`.
   - Após a leitura, o conteúdo do DataFrame `df` é exibido, o que ajuda a visualizar as primeiras linhas dos dados.

2. **Leitura do Arquivo 'precos.csv'**:
   - O arquivo 'precos.csv' também é carregado com o `pd.read_csv()`, mas desta vez a primeira coluna é definida como o índice do DataFrame (`index_col=0`).
   - A função `sample(30)` é usada para exibir uma amostra aleatória de 30 linhas do DataFrame `df_precos`, permitindo uma inspeção mais aleatória e diversificada dos dados carregados.

In [ ]:
# Lê o arquivo CSV 'precos.csv', definindo a primeira coluna como índice (index_col=0).
df_precos = pd.read_csv('precos.csv', index_col=0)

# Exibe uma amostra aleatória de 30 linhas do DataFrame df_precos.
df_precos.sample(30)


,codigoFipe,marca,modelo,anoModelo,mesReferencia,anoReferencia,valor
365105,035017-6,Porsche,968 Coupe 3.0 16V,1995,3,2014,60654.0
167561,015071-1,Hyundai,i30 2.0 16V 145cv 5p Aut.,2022,2,2013,58620.0
120367,003351-0,Ford,Fiesta SE 1.6 16V Flex 5p,2019,6,2020,47022.0
202912,018016-5,Kia Motors,Carnival 2.5 V6,2006,5,2019,32306.0
268922,022074-4,Mitsubishi,Pajero TR4 2.0/ 2.0 Flex 16V 4x4 Mec.,2015,7,2022,66123.0
319562,024030-3,Peugeot,206 Passion 1.6,2001,10,2015,11555.0
25694,008122-1,Audi,Q7 4.2 V8 40V 350cv Quattro Tiptronic,2010,4,2015,185664.0
95668,031048-4,Ferrari,F8 Spider 3.9 V8 Bi-Turbo 720cv,2022,5,2021,4700000.0
345728,024145-8,Peugeot,207 XR 1.4 Flex 8V 3p,2022,10,2011,32500.0
225644,020023-9,Mazda,Protegé Aut.,2000,10,2004,23521.0


# Pré processamento

In [ ]:
# Cria a coluna 'RISCO', calculando o risco como o valor da indenização dividido pela exposição.
df['RISCO'] = df['INDENIZACAO'] / df['EXPOSICAO']

# Filtra o DataFrame para manter apenas as linhas onde o valor da exposição é maior que 0.
df = df[df['EXPOSICAO'] > 0]

# Cria a coluna 'IDADE' calculando a idade com base na data de nascimento (assumindo que 'DATA_NASC' está no formato YYYYMMDD).
# A operação converte a coluna 'DATA_NASC' para ano, subtrai esse ano de 2020 para calcular a idade.
df['IDADE'] = 2020 - (df['DATA_NASC']/10000).astype(int)


C:\Users\angel\AppData\Local\Temp\ipykernel_32148\1652540542.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



O código realiza duas operações principais no DataFrame `df_precos`:

1. **Seleção de Colunas**:
   - O DataFrame é filtrado para manter apenas as colunas `codigoFipe` e `valor`. Essa operação simplifica o conjunto de dados, mantendo apenas as informações essenciais para o restante da análise.

2. **Renomeação de Coluna**:
   - A coluna `codigoFipe` é renomeada para `COD_MODELO`. Isso melhora a legibilidade e padroniza os nomes das colunas, tornando-os mais claros e adequados ao contexto de análise.

In [ ]:
# Seleciona as colunas 'codigoFipe' e 'valor' do DataFrame df_precos.
df_precos = df_precos[['codigoFipe', 'valor']]

# Renomeia a coluna 'codigoFipe' para 'COD_MODELO' para padronizar os nomes das colunas.
df_precos = df_precos.rename(columns={'codigoFipe': 'COD_MODELO'})

# Exibe o DataFrame df_precos com as modificações feitas.
df_precos


,COD_MODELO,valor
0,038003-2,13041.0
1,038001-6,52339.0
2,038003-2,12423.0
3,038002-4,31067.0
4,038002-4,26381.0
...,...,...
544147,061001-1,24811.0
544148,061001-1,31503.0
544149,061001-1,23582.0
544150,061001-1,31990.0


In [ ]:
# Agrupa o DataFrame df_precos pela coluna 'COD_MODELO' e calcula a média dos valores para cada grupo.
df_precos = df_precos.groupby('COD_MODELO').mean()

# Restaura o índice do DataFrame, criando um índice numérico padrão após o agrupamento.
df_precos.reset_index(inplace=True)

# Ordena o DataFrame df_precos pelos valores da coluna 'COD_MODELO'.
df_precos.sort_values('COD_MODELO')

# Exibe as primeiras linhas do DataFrame df_precos para inspeção.
df_precos.head()


,COD_MODELO,valor
0,001001-4,18014.75
1,001002-2,14847.75
2,001003-0,14325.25
3,001004-9,10084.00
4,001005-7,12045.00


In [298]:
df_precos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5283 entries, 0 to 5282
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   COD_MODELO  5283 non-null   object 
 1   valor       5283 non-null   float64
dtypes: float64(1), object(1)
memory usage: 82.7+ KB


In [299]:
df = pd.merge(df, df_precos, how='left')

In [300]:
df

,CHAVE,TIPO_PROD,COBERTURA,COD_MODELO,ANO_MODELO,COD_TARIF,REGIAO,TIPO_FRANQ,VAL_FRANQ,IS_CASCO,...,DATA_NASC,TEMPO_HAB,UTILIZACAO,CEP_PER,EXPOSICAO,OCORRENCIAS,INDENIZACAO,RISCO,IDADE,valor
0,2870169,1,1,001228-9,2007,10,9,2,3692.0,20121.0,...,19600126,0,2,12216000,0.295890,0,0.0,0.000000,60,26449.500000
1,479743,1,1,014094-5,2018,10,11,2,2681.0,70675.0,...,19850328,0,2,1231000,0.345205,0,0.0,0.000000,35,88058.323077
2,2201308,1,1,004347-8,2010,10,11,2,3030.0,23208.0,...,19390511,0,2,9720000,0.309589,0,0.0,0.000000,81,NaN
3,3132957,1,1,005190-0,2008,10,2,1,1546.0,19373.0,...,19730714,0,2,95330000,0.534247,0,0.0,0.000000,47,21630.693548
4,3474671,2,1,015095-9,2017,10,21,1,2549.0,56783.0,...,19971002,0,1,46430000,0.572603,0,0.0,0.000000,23,62461.885965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3032869,3971983,2,1,999999-9,2019,23,2,1,3899.0,142301.0,...,19791231,0,0,96180000,0.400000,0,0.0,0.000000,41,NaN
3032870,2901054,2,1,001461-3,2019,10,20,1,1718.0,33937.0,...,19800529,0,1,29018000,0.128767,1,7991.0,62057.765957,40,65747.250000
3032871,2820666,1,1,022112-0,2013,17,11,1,5921.0,124514.0,...,19550202,0,2,4116000,0.172603,0,0.0,0.000000,65,176889.615385
3032872,4297186,1,1,025054-6,2014,10,2,1,1890.0,20742.0,...,19820218,0,2,97050000,0.553425,0,0.0,0.000000,38,27193.906615


# Análise Exploratória

## Gráficos

In [306]:
def group_bar_plot(grouped, var, observed_column=None, predicted_column=None, max_values=None):
    grp = grouped
    if max_values:
        max_values = min(max_values, len(grp))
        grp = grp[:max_values]

    # Criar o gráfico de barra e linha
    fig = go.Figure()

    # Gráfico de barra para soma das exposições
    fig.add_trace(go.Bar(
        x=grp[var],
        y=grp['EXP'],
        name='Barra: Exposição (Texto: Ocorrências)',
        text=grp['OCR'],
    ))

    if observed_column:
        # Linha para prêmio de risco observado
        fig.add_trace(go.Scatter(
            x=grp[var],
            y=grp[observed_column],
            mode='lines+markers',
            name='Prêmio de Risco Observado',
            yaxis='y2',
        ))

    if predicted_column:
        # Linha para prêmio de risco estimado
        fig.add_trace(go.Scatter(
            x=grp[var],
            y=grp[predicted_column],
            mode='lines+markers',
            name='Prêmio de Risco Estimado',
            yaxis='y2'
        ))

    # Layout para eixos duplos
    fig.update_layout(
        title=f"Análise por grupo: {var}",
        xaxis=dict(title=var, type='category', tickmode='linear'),
        yaxis_title="Soma das Exposições",
        yaxis2=dict(title="Prêmio de Risco", overlaying='y', side='right', rangemode="tozero"),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        barmode='group',
    )

    fig.show()

def group_scatter_plot(grouped, var, observed_column=None, predicted_column=None, show_ocr=False):
    fig = go.Figure()

    # Gráfico de linha para eixo x contínuo
    fig.add_trace(go.Scatter(
        x=grouped[var],
        y=grouped[observed_column],
        text=None,
        name='Prêmio de Risco Observado',
        mode='lines+markers+text',
        textposition='top center',
        marker=dict(size=300 * grouped['EXP'] / grouped['EXP'].sum(), opacity=1.0)
    ))

    if predicted_column:
        # Linha para prêmio de risco estimado
        fig.add_trace(go.Scatter(
            x=grouped[var],
            y=grouped[predicted_column],
            mode='lines+markers',
            name='Prêmio de Risco Estimado',
        ))

    # Layout para eixos duplos
    fig.update_layout(
        title=f"Análise por grupo: {var}",
        yaxis=dict(title="Prêmio de Risco", rangemode="tozero"),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )

    fig.show()

def group_bar_plot_categorical(var_name, max_values=20, prediction = False):
    # Só use variáveis categóricas nessa variável
    GROUP_VAR = var_name # 'SEXO', 'COBERTURA', ...

    # Agrupar por variável e calcular as somas e métricas

    if prediction:
        grouped = df.groupby(GROUP_VAR).agg(
            EXP=('EXPOSICAO', 'sum'),
            OCR=('OCORRENCIAS', 'sum'),
            IND=('INDENIZACAO', 'sum'),
            PRED=('PRED_EXP', 'sum'),
        ).reset_index()
    else:
        grouped = df.groupby(GROUP_VAR).agg(
            EXP=('EXPOSICAO', 'sum'),
            OCR=('OCORRENCIAS', 'sum'),
            IND=('INDENIZACAO', 'sum'),
        ).reset_index()

    # Ordena por quem tem mais exposição
    grouped = grouped.sort_values('EXP', ascending=False)

    # Calcular o prêmio de risco observado
    if prediction:
        grouped['RISCO_GRUPO'] = grouped['IND'] / grouped['EXP']
        grouped['ESTIMADO_GRUPO'] = grouped['PRED'] / grouped['EXP']
    else:
        grouped['RISCO_GRUPO'] = grouped['IND'] / grouped['EXP']

    print(grouped.head())

    group_bar_plot(
        grouped,
        var=GROUP_VAR,
        observed_column='RISCO_GRUPO',
        predicted_column='ESTIMADO_GRUPO' if prediction else None,
        max_values=max_values
    )

def group_bar_plot_ordinal(var_name, max_values=20, prediction = False):
    # Só use variáveis ordinais nessa variável
    GROUP_VAR = var_name # TEMPO_HAB IDADE?

    # Agrupar por variável e calcular as somas e métricas
    if prediction:
        grouped = df.groupby(GROUP_VAR).agg(
            EXP=('EXPOSICAO', 'sum'),
            OCR=('OCORRENCIAS', 'sum'),
            IND=('INDENIZACAO', 'sum'),
            PRED=('PRED_EXP', 'sum'),
        ).reset_index()
    else:
        grouped = df.groupby(GROUP_VAR).agg(
            EXP=('EXPOSICAO', 'sum'),
            OCR=('OCORRENCIAS', 'sum'),
            IND=('INDENIZACAO', 'sum'),
        ).reset_index()

    # Calcular o prêmio de risco observado
    if prediction:
        grouped['RISCO_GRUPO'] = grouped['IND'] / grouped['EXP']
        grouped['ESTIMADO_GRUPO'] = grouped['PRED'] / grouped['EXP']
    else:
        grouped['RISCO_GRUPO'] = grouped['IND'] / grouped['EXP']

    print(grouped.head())

    group_bar_plot(
        grouped,
        var=GROUP_VAR,
        observed_column='RISCO_GRUPO',
        predicted_column='ESTIMADO_GRUPO' if prediction else None,
        max_values=max_values
    )

def group_bar_plot_continuous(var_name, max_values=20, prediction = False):
    # Só use variáveis contínuas nessa variável
    GROUP_VAR = var_name # VAL_FRANQ
    GROUP = GROUP_VAR + '_GRUPO'

    # Faixas da variável com igual número de apólices/contratos
    QCUT_BINS = 10

    if prediction:
        df_cut = df[[GROUP_VAR, 'EXPOSICAO', 'OCORRENCIAS', 'INDENIZACAO', 'PRED_EXP']].copy()
    else:
        df_cut = df[[GROUP_VAR, 'EXPOSICAO', 'OCORRENCIAS', 'INDENIZACAO']].copy()
        
    df_cut[GROUP] = pd.qcut(df[GROUP_VAR], QCUT_BINS, duplicates='drop')

    # Agrupar por variável e calcular as somas e métricas
    if prediction:
        grouped = df_cut.groupby(GROUP_VAR + '_GRUPO', observed=False).agg(
            MEAN=(GROUP_VAR, 'mean'),
            EXP=('EXPOSICAO', 'sum'),
            OCR=('OCORRENCIAS', 'sum'),
            IND=('INDENIZACAO', 'sum'),
            PRED=('PRED_EXP', 'sum'),
        ).reset_index().rename(columns={'MEAN': GROUP_VAR + '_MEDIA'})
    else:
        grouped = df_cut.groupby(GROUP_VAR + '_GRUPO', observed=False).agg(
            MEAN=(GROUP_VAR, 'mean'),
            EXP=('EXPOSICAO', 'sum'),
            OCR=('OCORRENCIAS', 'sum'),
            IND=('INDENIZACAO', 'sum'),
        ).reset_index().rename(columns={'MEAN': GROUP_VAR + '_MEDIA'})

    # Transforma os valores da categoria em string para compatabilidade com o Plotly
    categories_as_str = grouped[GROUP].cat.categories.astype(str)
    grouped[GROUP] = grouped[GROUP].cat.rename_categories(categories_as_str)

    # Calcular o prêmio de risco observado e estimado
    if prediction:
        grouped['RISCO_GRUPO'] = grouped['IND'] / grouped['EXP']
        grouped['ESTIMADO_GRUPO'] = grouped['PRED'] / grouped['EXP']
    else:
        grouped['RISCO_GRUPO'] = grouped['IND'] / grouped['EXP']

    print(grouped.head())

    group_bar_plot(
        grouped,
        var=GROUP,
        observed_column='RISCO_GRUPO',
        predicted_column='ESTIMADO_GRUPO' if prediction else None,
        max_values=max_values
    )

def group_bar_plot_all(var_name, type, max_values=20, prediction = False):
    if type == 'categorical':
        group_bar_plot_categorical(var_name, max_values, prediction)
    elif type == 'ordinal':
        group_bar_plot_ordinal(var_name, max_values, prediction)
    elif type == 'continuous':
        group_bar_plot_continuous(var_name, max_values, prediction)
    

In [307]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3032874 entries, 0 to 3032873
Data columns (total 22 columns):
 #   Column       Dtype  
---  ------       -----  
 0   CHAVE        int64  
 1   TIPO_PROD    int64  
 2   COBERTURA    int64  
 3   COD_MODELO   object 
 4   ANO_MODELO   int64  
 5   COD_TARIF    object 
 6   REGIAO       int64  
 7   TIPO_FRANQ   int64  
 8   VAL_FRANQ    float64
 9   IS_CASCO     float64
 10  CLAS_BONUS   int64  
 11  SEXO         object 
 12  DATA_NASC    int64  
 13  TEMPO_HAB    int64  
 14  UTILIZACAO   int64  
 15  CEP_PER      int64  
 16  EXPOSICAO    float64
 17  OCORRENCIAS  int64  
 18  INDENIZACAO  float64
 19  RISCO        float64
 20  IDADE        int32  
 21  valor        float64
dtypes: float64(6), int32(1), int64(12), object(3)
memory usage: 497.5+ MB


- **`name`**: O nome da variável.
- **`type`**: O tipo da variável, que pode ser uma das opções:
  - **`categorical`**: Variáveis que têm categorias distintas (como "SEXO" ou "TIPO_PROD").
  - **`ordinal`**: Variáveis com uma ordem entre os valores (como "ANO_MODELO" ou "IDADE").
  - **`continuous`**: Variáveis com valores contínuos, geralmente numéricos (como "VAL_FRANQ" ou "valor").
- **`max_values`** (opcional): Para variáveis do tipo `categorical` ou `ordinal`, indica o número máximo de valores únicos que a variável pode ter.

In [ ]:
# Define uma lista de dicionários, onde cada dicionário representa uma variável com informações sobre seu nome e tipo.
variables = [
    {"name": "TIPO_PROD", "type": "categorical"},
    {"name": "COBERTURA", "type": "categorical"},
    {"name": "COD_MODELO", "type": "categorical", "max_values": 100},
    {"name": "ANO_MODELO", "type": "ordinal", "max_values": 70},
    {"name": "COD_TARIF", "type": "categorical"},
    {"name": "REGIAO", "type": "categorical"},
    {"name": "TIPO_FRANQ", "type": "categorical"},
    {"name": "VAL_FRANQ", "type": "continuous"},
    {"name": "IS_CASCO", "type": "continuous"},
    {"name": "CLAS_BONUS", "type": "ordinal"},
    {"name": "SEXO", "type": "categorical"},
    {"name": "IDADE", "type": "ordinal", "max_values": 70},
    {"name": "TEMPO_HAB", "type": "ordinal"},
    {"name": "UTILIZACAO", "type": "categorical"},
    {"name": "CEP_PER", "type": "categorical", "max_values": 100},
    {"name": "valor", "type": "continuous"}
]

#### Variáveis

In [309]:
for variable in variables:
    group_bar_plot_all(variable['name'], variable['type'], max_values= variable['max_values'] if 'max_values' in variable else 20)

   TIPO_PROD            EXP    OCR          IND  RISCO_GRUPO
0          1  585095.690411  57422  386789719.0   661.070873
1          2  580205.676712  38842  310151414.0   534.554256


   COBERTURA           EXP    OCR          IND  RISCO_GRUPO
0          1  1.165141e+06  96256  696923509.0   598.145371
2          5  1.404329e+02      8      17624.0   125.497678
1          4  2.024658e+01      0          0.0     0.000000


     COD_MODELO           EXP   OCR         IND  RISCO_GRUPO
2749   015088-6  18263.542466  1711   8899053.0   487.257771
571    002111-3  17694.597260  1712  14879890.0   840.928436
1402   004424-5  16339.693151  1584   8788680.0   537.873014
970    003408-8  13704.057534  1429   6987924.0   509.916423
1742   005275-2  12528.780822   733   3490685.0   278.613302


   ANO_MODELO       EXP  OCR  IND  RISCO_GRUPO
0        1950  0.309589    0  0.0          0.0
1        1951  0.243836    0  0.0          0.0
2        1953  0.586301    0  0.0          0.0
3        1954  0.586301    0  0.0          0.0
4        1957  0.586301    0  0.0          0.0


   COD_TARIF            EXP    OCR          IND  RISCO_GRUPO
0         10  853805.008219  70116  416061821.0   487.303093
12        22   95090.268493   9593   95017481.0   999.234543
13        23   55009.520548   4960   76177950.0  1384.813924
1         11   53407.087671   3456   47787919.0   894.786087
2        14A   41808.164384   3929   17108974.0   409.225668


...

   IDADE          EXP  OCR        IND  RISCO_GRUPO
0     19    84.523288   11    60917.0   720.712619
1     20   694.402740   93   669155.0   963.641071
2     21  1470.232877  196  1242680.0   845.226644
3     22  2345.268493  293  2000899.0   853.164150
4     23  3645.841096  416  2628599.0   720.985619


   TEMPO_HAB           EXP    OCR          IND  RISCO_GRUPO
0          0  1.137716e+06  96219  696471756.0    612.16688
1          1  1.452055e-01      0          0.0      0.00000
2          2  2.328767e-01      0          0.0      0.00000
3          3  4.136986e-01      0          0.0      0.00000
4          4  1.284932e+00      0          0.0      0.00000


   UTILIZACAO            EXP    OCR          IND  RISCO_GRUPO
2           2  601367.832877  54536  341235074.0   567.431537
1           1  305859.698630  16468  112498094.0   367.809471
0           0  225868.471233  21955  223626116.0   990.072296
3           3   32205.364384   3305   19581849.0   608.030661


        CEP_PER          EXP  OCR        IND  RISCO_GRUPO
93357  22793000  2031.063014  147  1338866.0   659.194713
73523  15104000  1951.654795  192  1092116.0   559.584617
93213  22790000  1748.328767  119   886268.0   506.922964
95399  24220000  1521.304110   82   609293.0   400.507036
95439  24230000  1303.942466   79   592850.0   454.659631


                       valor_GRUPO   valor_MEDIA            EXP   OCR  \
0  (3137.8239999999996, 27300.183]  21595.031149  100130.736986  5224   
1           (27300.183, 33473.023]  30842.774336  100932.657534  6162   
2            (33473.023, 40300.78]  36285.824915   99564.638356  6708   
3            (40300.78, 45114.451]  43167.191499  100186.832877  7836   
4           (45114.451, 49981.912]  47298.507989   97454.389041  8613   

          IND  RISCO_GRUPO  
0  26200413.0   261.662041  
1  30677789.0   303.943141  
2  35506553.0   356.618108  
3  44389923.0   443.071427  
4  47840413.0   490.900548  


# Usando o CatBoost

1. **Definição das Variáveis**:
   - **`features`**: Lista com as variáveis independentes que serão usadas no treinamento do modelo. Essas variáveis servirão como entradas para prever a variável dependente (`RISCO`).
   - **`cat_features`**: Lista das variáveis categóricas que precisam de tratamento especial (como codificação) antes de serem usadas em um modelo.
   - **`target`**: Define a variável dependente, ou alvo, que será prevista pelo modelo. Neste caso, a variável é `RISCO`.

2. **Divisão dos Dados**:
   - O DataFrame `df` é dividido em dois conjuntos: `df_train` (dados de treinamento) e `df_test` (dados de teste). A função `train_test_split` realiza essa divisão de forma aleatória, com uma semente fixa (`random_state=0`) para garantir a reprodutibilidade do processo.
   - **`df_train.head()`**: Exibe as primeiras linhas do conjunto de dados de treinamento para inspecionar os dados.

In [ ]:
# Define a lista de variáveis independentes (features) usadas para o treinamento do modelo.
features = ['TIPO_PROD', 'COBERTURA', 'COD_TARIF', 'TIPO_FRANQ', 'VAL_FRANQ', 'IS_CASCO', 'CLAS_BONUS', 'SEXO', 'IDADE', 'UTILIZACAO', 'valor', 'TEMPO_HAB', 'REGIAO', 'CEP_PER']

# Define a lista de variáveis categóricas que serão tratadas separadamente.
cat_features = ['TIPO_PROD', 'COBERTURA', 'COD_TARIF', 'TIPO_FRANQ', 'CLAS_BONUS', 'SEXO', 'IDADE', 'UTILIZACAO', 'TEMPO_HAB', 'REGIAO', 'CEP_PER']

# Define a variável dependente ou alvo (target), que é a variável a ser prevista.
target = 'RISCO'

# Divide o DataFrame df em dois conjuntos: treino (df_train) e teste (df_test).
# A divisão é feita de forma aleatória com uma semente fixa para garantir reprodutibilidade (random_state=0).
df_train, df_test = train_test_split(df, random_state=0)

# Exibe as primeiras linhas do DataFrame de treino para inspeção.
df_train.head()


,CHAVE,TIPO_PROD,COBERTURA,COD_MODELO,ANO_MODELO,COD_TARIF,REGIAO,TIPO_FRANQ,VAL_FRANQ,IS_CASCO,...,DATA_NASC,TEMPO_HAB,UTILIZACAO,CEP_PER,EXPOSICAO,OCORRENCIAS,INDENIZACAO,RISCO,IDADE,valor
281026,2543296,2,1,015097-5,2018,10,16,1,2217.0,61901.0,...,19580419,0,0,36016210,0.586301,0,0.0,0.0,62,65612.956140
1686009,315621,2,1,002111-3,2016,10,7,1,3107.0,73274.0,...,19890831,0,0,80035100,0.353425,0,0.0,0.0,31,95921.248322
2981190,1900493,1,1,004426-1,2015,10,13,1,1663.0,41810.0,...,19840109,0,2,14887000,0.463014,0,0.0,0.0,36,NaN
2192170,2336619,1,1,008203-1,2017,10,11,2,7835.0,89230.0,...,19911205,0,2,7080000,0.586301,0,0.0,0.0,29,111734.135802
2622596,3896445,1,1,004360-5,2011,10,13,1,1377.0,19784.0,...,19830504,0,2,14095000,0.380822,0,0.0,0.0,37,35425.000000


In [ ]:
# Cria um Pool de dados para o conjunto de treinamento. O Pool é uma estrutura de dados específica usada pelo CatBoost.
# Ele contém as variáveis independentes (features), o alvo (target), as variáveis categóricas (cat_features),
# e o peso de cada instância, dado pela coluna 'EXPOSICAO'.
train_pool = Pool(df_train[features], label=df_train[target],
                  cat_features=cat_features, weight=df_train['EXPOSICAO'])

# Cria um Pool de dados para o conjunto de teste, com a mesma estrutura do conjunto de treinamento.
# Esse Pool será usado para avaliar o desempenho do modelo.
test_pool = Pool(df_test[features], label=df_test[target],
                 cat_features=cat_features, weight=df_test['EXPOSICAO'])


In [ ]:
cb_tweedie = CatBoostRegressor(
    loss_function='Tweedie:variance_power=1.5',    # Função de perda Tweedie
    n_estimators=500,                             # Reduzindo o número de estimadores
    depth=6,                                      # Reduzindo a profundidade das árvores
    learning_rate=0.1,                            # Aumentando a taxa de aprendizado
    l2_leaf_reg=3,                                # Regularização L2
    bagging_temperature=0.8,                       # Ajustando a aleatoriedade do bagging
    random_strength=1.5,                          # Força de aleatoriedade na divisão dos nós
    min_data_in_leaf=10,                          # Número mínimo de dados em cada folha
    task_type='CPU',                               # Usando CPU, se disponível
)

cb_tweedie.fit(train_pool, logging_level='Debug')

Features checksum calculation time: 0.03028554254
Create new LearnProgress
Fold: Use owned online single ctrs
Fold: Use owned online single ctrs
Fold: Use owned online single ctrs
Fold: Use owned online single ctrs
Mem usage: Before start train: 3762798592

IS_CASCO, bin=201 score 605519.7177
{UTILIZACAO} pr0 tb0 type1, border=5 score 631099.9306
{UTILIZACAO, TEMPO_HAB} pr0 tb0 type1, border=7 score 635410.141
{TIPO_FRANQ} pr0 tb0 type1, border=2 score 639221.3523
IS_CASCO, bin=85 score 642180.9089
IS_CASCO, bin=240 score 650704.6814
0:	learn: 1081.7256256	total: 1.13s	remaining: 9m 21s

IS_CASCO, bin=208 score 549577.8502
{UTILIZACAO} pr0 tb0 type1, border=6 score 576703.6803
TIPO_PROD, value=1 score 576353.043
{COD_TARIF} pr0 tb0 type1, border=2 score 584803.4079
{COD_TARIF, TEMPO_HAB} pr0 tb0 type1, border=9 score 588044.7291
{COD_TARIF} pr1 tb0 type0, border=3 score 590034.855
1:	learn: 979.6081041	total: 2.31s	remaining: 9m 36s

IS_CASCO, bin=206 score 498570.8227
{UTILIZACAO} pr0

In [ ]:
# Chama o método `get_all_params()` do modelo `cb_tweedie` para obter todos os parâmetros atuais do modelo.
cb_tweedie.get_all_params()

{'nan_mode': 'Min',
 'eval_metric': 'Tweedie:variance_power=1.5',
 'combinations_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=1:TargetBorderType=MinEntropy:Prior=0/1:Prior=0.5/1:Prior=1/1',
  'Counter:CtrBorderCount=15:CtrBorderType=Uniform:Prior=0/1'],
 'iterations': 500,
 'sampling_frequency': 'PerTree',
 'fold_permutation_block': 0,
 'leaf_estimation_method': 'Newton',
 'random_score_type': 'NormalWithModelSizeDecrease',
 'counter_calc_method': 'SkipTest',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'ctr_leaf_count_limit': 18446744073709551615,
 'bayesian_matrix_reg': 0.10000000149011612,
 'one_hot_max_size': 2,
 'eval_fraction': 0,
 'force_unit_auto_pair_weights': False,
 'l2_leaf_reg': 3,
 'random_strength': 1.5,
 'rsm': 1,
 'boost_from_average': False,
 'max_ctr_complexity': 4,
 'model_size_reg': 0.5,
 'simple_ctr': ['Borders:CtrBord

In [ ]:
# Faz previsões usando o modelo `cb_tweedie` no conjunto de dados de teste (`test_pool`).
# O resultado da previsão é armazenado na variável `cb_tweedie_pred`.
cb_tweedie_pred = cb_tweedie.predict(test_pool)

In [317]:
weights = df['EXPOSICAO']

In [ ]:
# Faz previsões usando o modelo `cb_tweedie` no conjunto de dados de teste, com base nas variáveis independentes (`features`).
# O resultado da previsão é armazenado na variável `predict`.
predict = cb_tweedie.predict(df_test[features])

# Armazena os valores reais da variável dependente (`RISCO`) no conjunto de teste.
risco = df_test[target]

# Armazena os pesos de cada instância no conjunto de teste, provenientes da coluna 'EXPOSICAO'.
test_weights = df_test['EXPOSICAO']

# Calcula o erro usando a função de perda `mean_tweedie_deviance` com a potência 1.5,
# que é apropriada para a regressão Tweedie, considerando os valores reais (`y_true`), as previsões (`y_pred`),
# os pesos das instâncias e o parâmetro `power`.
error = mean_tweedie_deviance(
    y_true=risco,
    y_pred=predict,
    sample_weight=test_weights,
    power=1.5
)

# Exibe o erro calculado no conjunto de teste.
print(f'Erro teste: {error}')

Erro teste: 149.07725068057695


In [ ]:
# Faz previsões usando o modelo `cb_tweedie` no conjunto de dados de treinamento, com base nas variáveis independentes (`features`).
# O resultado da previsão é armazenado na variável `predict`.
predict = cb_tweedie.predict(df_train[features])

# Armazena os valores reais da variável dependente (`RISCO`) no conjunto de treinamento.
risco = df_train[target]

# Armazena os pesos de cada instância no conjunto de treinamento, provenientes da coluna 'EXPOSICAO'.
train_weights = df_train['EXPOSICAO']

# Calcula o erro usando a função de perda `mean_tweedie_deviance` com a potência 1.5,
# que é apropriada para a regressão Tweedie, considerando os valores reais (`y_true`), as previsões (`y_pred`),
# os pesos das instâncias e o parâmetro `power`.
error = mean_tweedie_deviance(
    y_true=risco,
    y_pred=predict,
    sample_weight=train_weights,
    power=1.5
)

# Exibe o erro calculado no conjunto de treinamento.
print(f'Erro train: {error}')


Erro train: 148.2131287834396


In [322]:
df['PREDITO'] = cb_tweedie.predict(df[features])
df['PRED_EXP'] = df['PREDITO'] * weights

### Análise de Qualidade

In [ ]:
# Itera sobre todas as variáveis definidas na lista `variables`.
# Para cada variável, chama a função `group_bar_plot_all()` para gerar um gráfico de barras para as diferentes categorias ou valores dessa variável.
# Os parâmetros incluem:
# - O nome da variável (`variable['name']`).
# - O tipo da variável (`variable['type']`).
# - O valor máximo para categorias ou ordinais (se especificado na variável, senão é usado 20 como padrão).
# - `prediction=True`, indicando que o gráfico será gerado para visualização de variáveis relacionadas à previsão.

for variable in variables:
    group_bar_plot_all(variable['name'], variable['type'], max_values= variable['max_values'] if 'max_values' in variable else 20, prediction = True)


   TIPO_PROD            EXP    OCR          IND          PRED  RISCO_GRUPO  \
0          1  585095.690411  57422  386789719.0  3.758507e+08   661.070873   
1          2  580205.676712  38842  310151414.0  2.973938e+08   534.554256   

   ESTIMADO_GRUPO  
0      642.374731  
1      512.566130  


   COBERTURA           EXP    OCR          IND          PRED  RISCO_GRUPO  \
0          1  1.165141e+06  96256  696923509.0  6.732306e+08   598.145371   
2          5  1.404329e+02      8      17624.0  1.370834e+04   125.497678   
1          4  2.024658e+01      0          0.0  1.341556e+02     0.000000   

   ESTIMADO_GRUPO  
0      577.810585  
2       97.614923  
1        6.626088  


     COD_MODELO           EXP   OCR         IND          PRED  RISCO_GRUPO  \
2749   015088-6  18263.542466  1711   8899053.0  9.210753e+06   487.257771   
571    002111-3  17694.597260  1712  14879890.0  1.434384e+07   840.928436   
1402   004424-5  16339.693151  1584   8788680.0  7.951717e+06   537.873014   
970    003408-8  13704.057534  1429   6987924.0  6.753843e+06   509.916423   
1742   005275-2  12528.780822   733   3490685.0  3.342124e+06   278.613302   

      ESTIMADO_GRUPO  
2749      504.324563  
571       810.633858  
1402      486.650351  
970       492.835296  
1742      266.755705  


   ANO_MODELO       EXP  OCR  IND         PRED  RISCO_GRUPO  ESTIMADO_GRUPO
0        1950  0.309589    0  0.0   171.282611          0.0      553.257993
1        1951  0.243836    0  0.0   306.565831          0.0     1257.264363
2        1953  0.586301    0  0.0   249.685082          0.0      425.864742
3        1954  0.586301    0  0.0   215.299136          0.0      367.215816
4        1957  0.586301    0  0.0  1122.147365          0.0     1913.942936


   COD_TARIF            EXP    OCR          IND          PRED  RISCO_GRUPO  \
0         10  853805.008219  70116  416061821.0  4.095242e+08   487.303093   
12        22   95090.268493   9593   95017481.0  9.022926e+07   999.234543   
13        23   55009.520548   4960   76177950.0  7.016176e+07  1384.813924   
1         11   53407.087671   3456   47787919.0  4.258135e+07   894.786087   
2        14A   41808.164384   3929   17108974.0  1.777682e+07   409.225668   

    ESTIMADO_GRUPO  
0       479.646026  
12      948.880082  
13     1275.447514  
1       797.297644  
2       425.199645  


...

   IDADE          EXP  OCR        IND          PRED  RISCO_GRUPO  \
0     19    84.523288   11    60917.0  6.092412e+04   720.712619   
1     20   694.402740   93   669155.0  4.744007e+05   963.641071   
2     21  1470.232877  196  1242680.0  1.076966e+06   845.226644   
3     22  2345.268493  293  2000899.0  1.547408e+06   853.164150   
4     23  3645.841096  416  2628599.0  2.370748e+06   720.985619   

   ESTIMADO_GRUPO  
0      720.796861  
1      683.178025  
2      732.513903  
3      659.800014  
4      650.260827  


   TEMPO_HAB           EXP    OCR          IND          PRED  RISCO_GRUPO  \
0          0  1.137716e+06  96219  696471756.0  6.730137e+08    612.16688   
1          1  1.452055e-01      0          0.0  1.463472e-02      0.00000   
2          2  2.328767e-01      0          0.0  5.682998e-02      0.00000   
3          3  4.136986e-01      0          0.0  3.987353e-02      0.00000   
4          4  1.284932e+00      0          0.0  1.298754e-01      0.00000   

   ESTIMADO_GRUPO  
0      591.548333  
1        0.100786  
2        0.244035  
3        0.096383  
4        0.101076  


   UTILIZACAO            EXP    OCR          IND          PRED  RISCO_GRUPO  \
2           2  601367.832877  54536  341235074.0  3.339532e+08   567.431537   
1           1  305859.698630  16468  112498094.0  1.117403e+08   367.809471   
0           0  225868.471233  21955  223626116.0  2.089162e+08   990.072296   
3           3   32205.364384   3305   19581849.0  1.863477e+07   608.030661   

   ESTIMADO_GRUPO  
2      555.322729  
1      365.331744  
0      924.946313  
3      578.623231  


        CEP_PER          EXP  OCR        IND          PRED  RISCO_GRUPO  \
93357  22793000  2031.063014  147  1338866.0  1.151919e+06   659.194713   
73523  15104000  1951.654795  192  1092116.0  9.085685e+05   559.584617   
93213  22790000  1748.328767  119   886268.0  8.195690e+05   506.922964   
95399  24220000  1521.304110   82   609293.0  6.124040e+05   400.507036   
95439  24230000  1303.942466   79   592850.0  5.472901e+05   454.659631   

       ESTIMADO_GRUPO  
93357      567.150674  
73523      465.537496  
93213      468.772805  
95399      402.551967  
95439      419.719546  


                       valor_GRUPO   valor_MEDIA            EXP   OCR  \
0  (3137.8239999999996, 27300.183]  21595.031149  100130.736986  5224   
1           (27300.183, 33473.023]  30842.774336  100932.657534  6162   
2            (33473.023, 40300.78]  36285.824915   99564.638356  6708   
3            (40300.78, 45114.451]  43167.191499  100186.832877  7836   
4           (45114.451, 49981.912]  47298.507989   97454.389041  8613   

          IND          PRED  RISCO_GRUPO  ESTIMADO_GRUPO  
0  26200413.0  2.527823e+07   261.662041      252.452235  
1  30677789.0  3.101252e+07   303.943141      307.259520  
2  35506553.0  3.528632e+07   356.618108      354.406123  
3  44389923.0  4.443023e+07   443.071427      443.473727  
4  47840413.0  4.681230e+07   490.900548      480.350902  


# Gerando a resposta do teste:

In [ ]:
df_test = pd.read_csv('test.csv')

In [ ]:
df_test['RISCO'] = df_test['INDENIZACAO'] / df_test['EXPOSICAO']
df_test = df_test[df_test['EXPOSICAO'] > 0]
df_test['IDADE'] = 2020 - (df_test['DATA_NASC']/10000).astype(int)

In [ ]:
df_test = pd.merge(df_test, df_precos, how='left')

In [ ]:
df_test['PREDITO'] = cb_tweedie.predict(df_test[features])

In [ ]:
df_resposta = df_test[['CHAVE', 'PREDITO']]
df_resposta.to_csv('resposta.csv', index=False)